In [1]:
import os
import csv
import nest_asyncio
import asyncio
from datetime import datetime, timedelta
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update
from telegram.ext import Application, CommandHandler, CallbackQueryHandler, MessageHandler, filters, CallbackContext

# Global Variables
TOKEN = "XXX"
GROUP_CHAT_ID = -XXX

In [2]:
# File to store availability data
CSV_FILE = "availability.csv"

# Default game time settings
DEFAULT_START_TIMES = ["7:30 PM", "7:45 PM", "8:00 PM", "8:15 PM", "8:30 PM", "8:45 PM", "9:00 PM", "9:15 PM", "9:30 PM"]
DEFAULT_END_TIMES = ["8:30 PM", "8:45 PM", "9:00 PM", "9:15 PM", "9:30 PM", "9:45 PM", "10:00 PM", "10:15 PM", "10:30 PM"]

# Store user selections temporarily
user_data = {}

In [3]:
# Start command
async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text(
        "I'm the Giant Rat Bot that makes all of the rules. 🐀\nUse /edit to set your schedule."
    )

In [4]:
# Free vs Busy button + response
async def edit(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [InlineKeyboardButton("FREE", callback_data="status_free"),
         InlineKeyboardButton("BUSY", callback_data="status_busy")]
    ]
    await update.message.reply_text(
        "What status d'ya wanna set?", reply_markup=InlineKeyboardMarkup(keyboard)
    )

async def handle_response(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    await query.answer()
    user_id = query.from_user.id
    username = query.from_user.username or query.from_user.first_name
    status = query.data.split("_")[1]
    timestamp_date = datetime.now().strftime("%Y-%m-%d")
    timestamp_time = datetime.now().strftime("%H:%M")
    user_data[user_id] = {
        "username": username,
        "status": status,
        "date": None,
        "start": None,
        "end": None,
        "timestamp_date": timestamp_date,
        "timestamp_time": timestamp_time
    }

    if status == "busy":
        await send_date_selection(query, busy=True)
    else:
        await send_date_selection(query, busy=False)

In [5]:
# Date selection query + response
async def send_date_selection(query, busy=False):
    """Send inline buttons for date selection."""
    today = datetime.today().strftime("%Y-%m-%d")
    keyboard = []
    for i in range(7):  # Show 7 upcoming days
        date = datetime.today() + timedelta(days=i)
        date_str = date.strftime("%Y-%m-%d")
        display_text = date.strftime("%b %d (%a)")
        if date_str == today:
            display_text = "TODAY"
        keyboard.append([InlineKeyboardButton(display_text, callback_data=f"date_{date_str}")])
    
    # today = datetime.today()
    # keyboard = []
    # for i in range(5):  # Show 5 upcoming days
    #     date_str = (today + timedelta(days=i)).strftime("%Y-%m-%d")
    #     keyboard.append([InlineKeyboardButton(date_str, callback_data=f"date_{date_str}")])

    if busy:
        await query.message.reply_text("Which day are you busy?", reply_markup=InlineKeyboardMarkup(keyboard))
    else:
        await query.message.reply_text("So when do you wanna game?", reply_markup=InlineKeyboardMarkup(keyboard))

async def handle_date_selection(update: Update, context: CallbackContext) -> None:
    """Handle user's date selection."""
    query = update.callback_query
    await query.answer()
    user_id = query.from_user.id
    selected_date = query.data.split("_")[1]
    user_data[user_id]["date"] = selected_date
    
    # Ask for start time selection
    if user_data[user_id]["status"] == "busy":
        await save_availability(user_id, query, busy=True)
    else:
        await send_time_selection(query, "start")

In [6]:
# Time selection query + response
async def send_time_selection(query, time_type):
    """Send inline buttons for start or end time selection."""
    times = DEFAULT_START_TIMES if time_type == "start" else DEFAULT_END_TIMES
    keyboard = [[InlineKeyboardButton(t, callback_data=f"{time_type}_{t}")] for t in times]
    await query.message.reply_text(
        f"Select {time_type} time:", reply_markup=InlineKeyboardMarkup(keyboard)
    )

async def handle_time_selection(update: Update, context: CallbackContext) -> None:
    """Handle user's time selection."""
    query = update.callback_query
    await query.answer()
    user_id = query.from_user.id
    time_type, selected_time = query.data.split("_")
    user_data[user_id][time_type] = selected_time
    
    if time_type == "start":
        await send_time_selection(query, "end")
    else:
        await save_availability(user_id, query)

In [7]:
# Save full availability response to CSV
async def save_availability(user_id, query, busy=False):
    data = user_data[user_id]
    existing_entries = []
    formatted_date = datetime.strptime(data["date"], "%Y-%m-%d").strftime("%b %d (%a)")

    # Read existing CSV data and filter out old entries for this user and date
    if os.path.exists(CSV_FILE):
        with open(CSV_FILE, "r", newline="") as file:
            reader = csv.reader(file)
            existing_entries = [row for row in reader if not (row[0] == data["username"] and row[1] == data["date"])]
        
    # Write the updated data (without old entries) back to CSV
    with open(CSV_FILE, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(existing_entries)  # Keep all unchanged entries
        writer.writerow([data["username"], data["date"], "BUSY" if busy else data["start"], "BUSY" if busy else data["end"], data["status"], data["timestamp_date"], data["timestamp_time"]])
    
    if busy:
        await query.message.reply_text(f"🚫 Marked as busy on {formatted_date}.")
    else:
        await query.message.reply_text(f"✅ Marked as available on {formatted_date} from {data['start']} to {data['end']}.")
        
    del user_data[user_id]

In [8]:
# Check availbility + ping unresponsive users
async def check_availability(update: Update, context: CallbackContext) -> None:
    chat_id = GROUP_CHAT_ID
    
    users_who_replied = set()
    all_users = set([user.id for user in await context.bot.get_chat_administrators(update.message.chat_id)])
    
    with open(CSV_FILE, "r") as file:
        reader = csv.reader(file)
        for row in reader:
            users_who_replied.add(row[0])
    
    silent_users = all_users - users_who_replied
    if silent_users:
        for user in silent_users:
            await context.bot.send_message(chat_id=update.message.chat_id, text=f"🐀: Hey @{user}, you wanna game or what? Submit your availability for tonight before it's too late!")

In [9]:
# Send final availbility
async def finalize_game_plan(update: Update, context: CallbackContext) -> None:
    """Send final game plan summary."""
    if not os.path.exists(CSV_FILE) or os.stat(CSV_FILE).st_size == 0:
        await context.bot.send_message(update.message.chat_id, "📭 No availability data yet! Use /edit to set your schedule.")
        return
        
    availability = {}
    with open(CSV_FILE, "r") as file:
        reader = csv.reader(file)
        for row in reader:
            date, start, end, status = row[1:]
            if status == "free":
                availability.setdefault(date, []).append(row[0], start)
    
    today = datetime.today().strftime("%Y-%m-%d")
    if today in availability and len(availability[today]) >= 3:
        await context.bot.send_message(update.message.chat_id, f"🎮 Game Night Confirmed!\n{len(availability[today])} rats: {players_list}\n🕗 Start Time: {start_time}.")
    else:
        await context.bot.send_message(update.message.chat_id, "❌ Not enough players for tonight. Looks like yer all on yer own.")

In [10]:
# async def get_chat_id(update: Update, context: CallbackContext) -> None:
#     """Sends the chat ID when the /chatid command is used."""
#     chat_id = update.message.chat_id
#     await update.message.reply_text(f"Chat ID: {chat_id}")

In [11]:
# Bot setup
async def main():
    nest_asyncio.apply()  # Allows async loops inside Jupyter
    application = Application.builder().token(TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("edit", edit))
    application.add_handler(CommandHandler("finalize", finalize_game_plan))
    application.add_handler(CallbackQueryHandler(handle_response, pattern="^status_"))
    application.add_handler(CallbackQueryHandler(handle_date_selection, pattern="^date_"))
    application.add_handler(CallbackQueryHandler(handle_time_selection, pattern="^(start|end)_"))

    await application.run_polling()

In [12]:
asyncio.create_task(main())

<Task pending name='Task-5' coro=<main() running at /var/folders/q0/f11k0by57hz_fs299ptxjl2w0000gn/T/ipykernel_25082/985870611.py:2>>